# Quickstart

This notebook serves as an introduction on getting started with pyCIMS. Before we dive in, make sure you have downloaded or cloned the pyCIMS git repo from https://gitlab.rcg.sfu.ca/mlachain/pycims_prototype/-/tree/master and have ensured that it is up-to-date

Now, we can get started with an example. 

## Import pyCIMS  & other packages
We need to make sure we can access the `pyCIMS` module. Since we are still developing the module, we can't install it from PyPI. Instead, we will make sure the module is in our PATH. The code below assumes you started the jupyter notebook from inside the `pycims_prototype/code/tutorial_examples` directory. 

In [ ]:
import os
import sys
import pprint as pp

sys.path.insert(0, os.path.abspath('../../')) # Assumes ipynb ran from pycims_prototype/code/model_validation
sys.path.insert(0, os.path.abspath('../../pyCIMS')) # Assumes ipynb ran from pycims_prototype/code/model_validation

import pyCIMS

We will also need to install the `pyxlsb` package. This is an optional dependency of `pyCIMS` that is required when our model descriptions are saved as `.xlsb` files.   

In [ ]:
!pip install pyxlsb

## Model Validator

Now that we have loaded the `pyCIMS` module we can start to use it. First, we will validate our model description using the `ModelValidator` class.

In [ ]:
model_description_file = '../../pyCIMS_model_description_Alberta_Validated.xlsb' # Assumes ipynb ran from pycims_prototype/code/tutorial_examples
model_validator = pyCIMS.ModelValidator(model_description_file)
model_validator.validate(verbose=True, raise_warnings=False)

For more information on using the `ModelValidator` class, checkout the [model validation tutorial](model_validation_instructions.ipynb). 

## Build the Model
There are two steps to building the model. First, we need to create a `ModelReader` object. This object reads in the model description Excel file and does some processessing to prepare the description to become a model. Then, this `ModelReader` object becomes the input to the all important `Model` object.

In [ ]:
# Model Reader
model_reader = pyCIMS.ModelReader(infile=model_description_file,
                                  sheet_map={'model': 'Model'},
                                  node_col='Node')

In [ ]:
# Model
model = pyCIMS.Model(model_reader)

## Run the pyCIMS Model

In [ ]:
model.run(max_iterations=5, show_warnings=False)

## Examine the Results

In [ ]:
# How has a fuel's price changed between years? 
import pprint as pp

price_of_interest = 'pyCIMS.Canada.Alberta.Electricity'
for year in model.years:
    pp.pprint(model.graph.nodes[price_of_interest][year]['Life Cycle Cost'])


In [ ]:
# How has marketshare changed between years ?
node_of_interest = 'pyCIMS.Canada.Alberta.Residential.Buildings.Floorspace.Lighting'
for year in model.years:
    print('***** {} *****'.format(year))
    for t, d in model.graph.nodes[node_of_interest][year]['technologies'].items():
        print("{}: {}%".format(t, round(d['total_market_share']*100)))
